<a href="https://colab.research.google.com/github/Shurui-Zhang/Deep_learning/blob/main/Lab4ex_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

In [ ]:
import torch
import torch.nn.functional as F #包含 torch.nn 库中所有函数 同时包含大量 loss 和 activation function
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

In [ ]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

# define baseline model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) 
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)  
        self.fc4 = nn.Linear(hidden_size3, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

In [ ]:
import torchbearer
import matplotlib.pyplot as plt
hidden1 = 15000
hidden2 = 15000
hidden3 = 14000
#hidden = 784

  

model = BaselineModel(784, hidden1, hidden2, hidden3, 10)


loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)


results_train = trial.run(epochs=10)# 

list_epoch = []
list_acc = []
list_loss = []
for i in range(10):#
  list_epoch.append(i)
  list_acc.append(results_train[i]['acc'])
  list_loss.append(results_train[i]['loss'])


plt.plot(list_epoch, list_acc, label='acc')
plt.xlabel("the number of iterations")
plt.ylabel("acc")
plt.legend(loc="upper left")
plt.show()

plt.plot(list_epoch, list_loss, label='loss')
plt.xlabel("the number of iterations")
plt.ylabel("loss")
plt.legend(loc="upper left")
plt.show()

# list_epoch_test = []
# list_acc_test = []
# list_loss_test = []
# for zzz in range (30):
#   results_test = trial.evaluate(data_key=torchbearer.TEST_DATA)
#   list_epoch_test.append(zzz)
#   list_acc_test.append(results_test['test_acc'])
#   list_loss_test.append(results_test['test_loss'])
# plt.plot(list_epoch_test, list_acc_test, label='acc')
# plt.xlabel("the number of iterations")
# plt.ylabel("acc")
# plt.legend(loc="upper left")
# plt.show()

# plt.plot(list_epoch_test, list_loss_test, label='loss')
# plt.xlabel("the number of iterations")
# plt.ylabel("loss")
# plt.legend(loc="upper left")
# plt.show()

results_test = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results_test)

#running为每个batch中的结果。acc为每次epoch中的平均值

RuntimeError: ignored